In [2]:
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime
import os
import requests
import json

# 初始化模型和其他資源
model = YOLO("yolov8n-pose.pt")
knn_model = joblib.load('knn_model.pkl')
knn_scaler = joblib.load('knn_scaler.pkl')
path = "./fall.mp4"
cap = cv2.VideoCapture(path)

# LINE Notify token
token = '5RYO1QtkRHIJ5UH9GmC8heZYJ30XBsnqkChyIg3lTO0'

# 伺服器端點的URL
server_url = 'http://192.168.24.94:8080/api/pictures'

# 將信心值加入關鍵點座標
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.boxes)):
        box_results = []
        for j in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            box_results.extend([x, y])
        combined_results.append(box_results)
    return combined_results

# LINE Notify傳送訊息
def send_line_notify(token, image_path=None):
    message = '有人跌倒了'
    headers = {"Authorization": "Bearer " + token}
    data = {'message': message}
    if image_path:
        with open(image_path, 'rb') as image_file:
            image_data = {'imageFile': image_file}
            response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data, files=image_data)
    else:
        response = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
    print(response.status_code)
    print(response.text)

# 發送跌倒事件到伺服器，並上傳圖片
def send_fall_event_to_server(timestamp, image_path=None):
    # 準備 JSON 數據
    data = {
        'event': 'fall_detected',
        'timestamp': timestamp
    }
    
    # 如果有圖片路徑，則讀取圖片並準備上傳
    files = None
    if image_path:
        files = {'image': open(image_path, 'rb')}
    
    # 發送 POST 請求
    headers = {'Content-Type': 'application/json'}
    response = requests.post(server_url, 
                             data=json.dumps(data), 
                             headers=headers, 
                             files=files)
    
    # 處理伺服器的回應
    if response.status_code == 200:
        print(f'事件成功發送: {timestamp}')
    else:
        print(f'事件發送失敗: {response.status_code}, {response.text}')


# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model.track(frame, conf=0.3)
    annotated_frame = results[0].plot()
    r = results[0]
    if len(results[0].boxes) >= 1:
        data = np.array(process_image(results))
        data = knn_scaler.transform(data)
        y_pred = knn_model.predict(data)
        print(y_pred)
        if y_pred[0] == 2:
            fall_current_time = current_time
            folder_name = 'Fall_img'
            os.makedirs(folder_name, exist_ok=True)
            time_text = fall_current_time.strftime("%Y-%m-%d %H:%M:%S")
            image_filename = fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg")
            cv2.imwrite(os.path.join(folder_name, image_filename), frame)
            print("output save")
            # 通知LINE，並將圖片送出
            #image_path = os.path.join(folder_name, image_filename)
            #send_line_notify(token, image_path=image_path)
            # 發送跌倒事件到伺服器
            send_fall_event_to_server(time_text)
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\User\anaconda3\envs\py_39_cv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.1.3 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\User\anaconda3\envs\py_39_cv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.1.3 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



0: 384x640 1 person, 211.9ms
Speed: 4.0ms preprocess, 211.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 204.0ms
Speed: 2.1ms preprocess, 204.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 195.9ms
Speed: 2.0ms preprocess, 195.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 206.0ms
Speed: 2.0ms preprocess, 206.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 197.6ms
Speed: 2.0ms preprocess, 197.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 196.9ms
Speed: 2.0ms preprocess, 196.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 196.8ms
Speed: 2.0ms preprocess, 196.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[1]

0: 384x640 1 person, 194.0ms
Speed: 2.0ms preprocess, 194.0ms inference, 1.

ConnectionError: HTTPConnectionPool(host='192.168.24.94', port=8080): Max retries exceeded with url: /api/pictures (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E4B3192D90>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。'))

: 